In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.part_number/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                data = {'dopartsearch': '1',
                        'partsearch[partnum][partsearch_007]': input_.loc[a, 'Part Number'],
                        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'Manufacturer'],
                        'partsearch[parttype][partsearch_007]': input_.loc[a, 'Type Code'],
                        'partsearch[do][partsearch_007]': '1',
                        'func': 'sendparttabsearch',
                        'giveback': json.dumps({}),
                        'api_json_request': '1'}

                b = 0
                while True:
                    b += 1
                    try:
                        check_url = input_.loc[a, 'Url']
                        
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))

                        if resp.status_code == 200:
                            dict_ = resp.json()
                            
                            if 'redirect_to_url' not in dict_ and not dict_['giveback'] and 'collected_javascript' in dict_:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
                            
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')
                
                # = = = = = = = = = = = = = = =

                list_list_note = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]') for tbody in list_tbody]
                list_list_note = [[etree.tostring(note).decode('utf-8') for note in list_note] if list_note else [] for list_note in list_list_note]
                list_list_note = [[re.sub(r'<.*?>', '', note) for note in list_note] for list_note in list_list_note]
                list_list_note = [[re.sub(r' +', ' ', note).strip() for note in list_note] for list_note in list_list_note]

                # = = = = = = = = = = = = = = =

                list_dict_note = []
                for list_note in list_list_note:
                    dict_note = {}
                    for i, note in enumerate(list_note):
                        dict_note[str(i)] = note
                
                    list_dict_note.append(dict_note)
                
                # = = = = = = = = = = = = = = =

                list_list_note = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]') for tbody in list_tbody]
                list_list_note = [[etree.tostring(text).decode('utf-8') for note in list_note for text in note.xpath('./span[@class="listing-footnote-text"]')] if list_note else [] for list_note in list_list_note]
                list_list_note = [[re.sub(r'<.*?>', '', note) for note in list_note] for list_note in list_list_note]
                list_list_note = [[re.sub(r' +', ' ', note).strip() for note in list_note] for list_note in list_list_note]

                # = = = = = = = = = = = = = = =

                list_dict_note_2 = []
                for list_note in list_list_note:
                    dict_note = {}
                    for i, note in enumerate(list_note):
                        dict_note[str(i)] = note
                
                    list_dict_note_2.append(dict_note)
    
                # = = = = = = = = = = = = = = =

                list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_type_code = [dict_['parttype'] for dict_ in list_dict]

                # = = = = = = = = = = = = = = =

                array_index_type_code = np.where(np.array(list_type_code) == input_.loc[a, 'Type Code'])

                # = = = = = = = = = = = = = = =

                list_part_code = [dict_['partkey'] for dict_ in list_dict]

                # = = = = = = = = = = = = = = =

                list_list_info = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]') for tbody in list_tbody]
                list_info = [list_info[0].xpath('./@href')[0].strip() if list_info else '' for list_info in list_list_info]
                list_info = [info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={part_code}&cc=0&pt={type_code}&Lennon=1''' for info, part_code, type_code in zip(list_info, list_part_code, list_type_code)]

                # = = = = = = = = = = = = = = =

                list_dict = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                array_index_part_number = np.where(np.array([dict_['partnumber'] for dict_ in list_dict]) == input_.loc[a, 'Part Number'])

                # = = = = = = = = = = = = = = =

                array_index_manufacturer = np.where(np.array([dict_['catalogname'] for dict_ in list_dict]) == input_.loc[a, 'Manufacturer'])

                # = = = = = = = = = = = = = = =

                array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

                # = = = = = = = = = = = = = = =

                if len(array_index_part) == 1:
                    index_part = array_index_part[0]
                else:
                    raise

                # = = = = = = = = = = = = = = =

                choose = list_dict[index_part]['paramdesc'].strip()

                # = = = = = = = = = = = = = = =

                list_list_text = [td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')]

                # = = = = = = = = = = = = = = =

                list_oem = []
                for list_text in list_list_text:
                    if list_text:
                        for text in list_text:
                            list_oem += text.split(',')
                        
                list_oem = [oem.strip() for oem in list_oem]
                    
                while '' in list_oem:
                    list_oem.remove('')
                
                oem = ';'.join(sorted(list(set(list_oem))))

                # = = = = = = = = = = = = = = =

                list_src = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
                dict_ = json.loads(list_src[0].xpath('./@value')[0]) if list_src else {}

                # = = = = = = = = = = = = = = =

                dict_src = {}
                if dict_:
                    list_src = dict_['Slots']
                    for i, src in enumerate(list_src):
                        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
        
                # = = = = = = = = = = = = = = =

                price = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
                price = etree.tostring(price[0]).decode('utf-8') if price else ''
                price = re.sub(r'<.*?>', '', price)
                price = re.sub(r' +', ' ', price).strip()
                dict_price = {'0': price}

                # = = = = = = = = = = = = = = =

                td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')[0]
                list_option = [etree.tostring(option).decode('utf-8') for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
                list_option = [re.sub(r'<.*?>', '', option) for option in list_option]
                list_option = [re.sub(r' +', ' ', option).strip() for option in list_option]
                if list_option:
                    dict_price['1'] = {}
                    for i, option in enumerate(list_option):
                        dict_price['1'][str(i)] = option

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Vehicle': '',
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Info': list_info[index_part],
                                         'Choose': choose,
                                         'Json_Price': json.dumps(dict_price),
                                         'Json_Note 1': json.dumps(list_dict_note[index_part]),
                                         'Json_Note 2': json.dumps(list_dict_note_2[index_part]),
                                         'Group': input_.loc[a, 'Group'],
                                         'Type Code': input_.loc[a, 'Type Code'],
                                         'Part Code': list_part_code[index_part],
                                         'JOIN_MPNTCPC': f'''{input_.loc[a, 'Manufacturer']};{input_.loc[a, 'Part Number']};{input_.loc[a, 'Type Code']};{list_part_code[index_part]}'''}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Check_Url'] = check_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{check_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.rockauto.com/en/parts/melling,30250S,timing+chain+&+component+kit,5756
[剩余数量：5] - [当前时间：19:03:26]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.rockauto.com/en/parts/melling,30250SX,timing+chain+&+component+kit,5756
[剩余数量：4] - [当前时间：19:03:27]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.https://www.rockauto.com/en/parts/melling,30460SA,timing+chain+&+component+kit,5756
[剩余数量：3] - [当前时间：19:03:27]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 8.https://www.rockauto.com/en/parts/melling,30470S,timing+chain+&+component+kit,5756
[剩余数量：2] - [当前时间：19:03:29]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://www.rockauto.com/en/parts/melling,30460SB,timing+chain+&+component+kit,5756
[剩余数量：1] - [当前时间：19:03:30]

[状态：ok，尝试次数：2] - 50.00% - crawler_1 > 4.https://www.rockauto.com/en/parts/melling,30430S,timing+chain+&+component+kit,5756
[剩余数量：0] - [当前时间：19:03:30]

[状态：ok，尝试次数：2] - 50.00% - crawler_1 > 5.https://www.rockauto.com/en/parts/melling,3

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4341.93it/s]

Done ~
